# MDE for PNA PENNY initial experiment

In [61]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sys

# Move two levels up (to the project root) and append the `src` folder
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Append src to sys.path
sys.path.append(src_path)

# Path for customer data analytics repo
base_path = '/Users/jordicremades/Documents/repos/customer-data-analytics/'
sys.path.insert(1, os.path.join(base_path, 'General/AB-tests/MDE'))
sys.path.insert(2, os.path.join(base_path, 'General/AB-tests'))

from src import query_engines

q = query_engines.QueryEngines()

# Imports for customer data analytics repo
from fenix.statistical_tests import relative_ttest as t_test
from fenix.run_experiment import ExperimentConfig, MDEConfig, ExperimentRunner
from fenix.statistical_tests.stattools import StatisticalResult
from fenix.tools.helpfuncs import mde_visualization
from fenix.tools.helpfuncs import adjusted_alpha_multiple_comparison

import transformations.transformation as tr
import fenix.visual.graphs
import fenix.tools.hashing as hashing

## Query

In [59]:
START_DATE  = "'2024-10-01'"
END_DATE  = "'2024-11-01'"
BUFFER = '60'

params = [
    {'name':'start_date', 'value': str(START_DATE)},
    {'name':'end_date', 'value': str(END_DATE)},
    {'name':'buffer', 'value': str(BUFFER)}
]

q.prepare_query('mde_pna.sql', params=params, to_load_file=None, load_from_to_load_file=None)

df = q.query_run_starburst()

Open the following URL in browser for the external authentication:
https://starburst.g8s-data-platform-prod.glovoint.com/oauth2/token/initiate/b9591ede6e89045b9e10d0dfd11285f6e797111e93b8db7697dbd6f3e6009ea2


In [77]:
# LTHO
df['ltho'] = np.random.choice(['Control', 'Variant'], size = len(df), p=[0.9,0.10])
ltho_variant = (df[df['ltho'] == 'Control']).copy()

# Cart experiment
ltho_variant['cart_experiment'] = np.random.choice(['Control', 'Variant'], size = len(ltho_variant), p=[0.5,0.5])
cart_variant = (cart_variant[cart_variant['cart_experiment'] == 'Variant']).copy()

# PNA experiment
cart_variant['pna_experiment'] = np.random.choice(['Control', 'Variant'], size = len(cart_variant), p=[0.5,0.5])
pna_variant = (cart_variant[cart_variant['pna_experiment'] == 'Variant']).copy()
pna_control = (cart_variant[cart_variant['pna_experiment'] == 'Control']).copy()


In [84]:
print('Number of customers: ', pna_variant['customer_id'].nunique())
print('Number of ordres: ', pna_variant['orders'].sum())
print('Number of products: ', pna_variant['products'].sum())

Number of customers:  6683
Number of ordres:  12895
Number of products:  147860


In [85]:
print('Number of customers: ', pna_control['customer_id'].nunique())
print('Number of ordres: ', pna_control['orders'].sum())
print('Number of products: ', pna_control['products'].sum())

Number of customers:  6633
Number of ordres:  12734
Number of products:  146736
